In [1]:
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
%matplotlib inline
# %matplotlib ipympl
import os
import numpy as np
import pynbody
from pynbody.plot import sph
import matplotlib.pylab as plt
import numba
import pickle

from snap_io import load_moria_sim_and_kicked, load_moria, load_kicked, load_sim

In [3]:
load=False

In [4]:
SIMNUMBER = 68002
kicked=True

In [5]:
resolution = 500
width = 10 # kpc
vmin=2e-1
vmax=5e-4;

In [6]:
# sim = load_moria(SIMNUMBER, 10)
# sim.properties.pop('boxsize')
# sim.properties

In [7]:
# sim.g['smooth'].mean()

In [8]:
# sim.g['smooth'] /= 2

In [9]:
# # pynbody.analysis.angmom.faceon(sim)
# pynbody.analysis.halo.center(sim)

In [10]:
# fig, ax = plt.subplots(figsize=(10,10))
# sph.image(sim.g, subplot=ax, qty="rho", resolution=500, units="g cm^-3",
#           width=10, log=False);
# time = '$t={%5.2f}$ Gyr' % sim.properties['time'].in_units("Gyr")
# ax.set_title(time, fontsize=12)

In [18]:
def gas_image(sim, width=width, resolution=resolution, vmin=vmin, vmax=vmax, noplot=False):
    sim.properties.pop('boxsize', None)
    sim.g['smooth'] /= 2
    pynbody.analysis.halo.center(sim, shrink_factor=0.9)
    img = sph.image(sim.g, qty="rho", resolution=resolution, units="g cm^-2",
              width=width, noplot=noplot, vmin=vmin, vmax=vmax)
              # filename="{}/{}_{:03d}".format(foldername, SIMNUMBER, i));
    return img

In [19]:
# foldername = "pngs_{}".format(SIMNUMBER)
# os.makedirs(foldername, exist_ok=True)

In [20]:
class GasImage:
    def __init__(self, img=None, time=0, snap=0):
        self.img = img
        self.time = time
        self.snap = snap
    @property
    def title(self):
        return '$t={:5.2f}$ Gyr, snap={:03d}'.format(self.time, self.snap)

In [21]:
@numba.jit
def precompute_imgs(snap_list):
#     gas_imgs = [GasImage()] * len(snap_list)
    gas_imgs = list()
    for i, sim in enumerate(snap_list):
        gas_imgs.append(GasImage(gas_image(sim, noplot=True), time=sim.properties['time'].in_units("Gyr"), snap=i))
#         gas_imgs[i].img = gas_image(sim) 
#         gas_imgs[i].time = sim.properties['time'].in_units("Gyr")
#         gas_imgs[i].snap = i
        print(i)
    return gas_imgs

In [22]:
pickle_name = {False: "imgs_sim%d.pickle" % SIMNUMBER, True: "imgs_sim%d_k.pickle" % SIMNUMBER}

In [23]:
if load:
    with open(pickle_name[kicked], "rb") as f:
        imgs = pickle.load(file=f)

In [24]:
if not load:
    snap_list = load_kicked(SIMNUMBER) if kicked else load_moria(SIMNUMBER)
    print(snap_list[0], len(snap_list))
    imgs = precompute_imgs(snap_list)
    with open(pickle_name[kicked], "wb") as f:
            pickle.dump(imgs, file=f)

<SimSnap "/home/michele/sim/MySimulations/Moria8Gyr_tidal/sim68002/out/snapshot_0001" len=1952204> 95
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [26]:
imgs[3].title

'$t= 8.39$ Gyr, snap=003'

In [27]:
# import joblib
# f = joblib.Parallel(n_jobs=-1, backend="threading")(joblib.delayed(precompute_imgs)(sim) for i, sim in enumerate(snap_list))

In [28]:
# w_axes = interactive(plot_axes, i=slider, continuous_update=False)
# hb = widgets.HBox([play, slider])
# widgets.VBox([hb, w])

In [29]:
def plot_gas(gas_img):
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(gas_img.img[::-1, :].view(np.ndarray), extent=(-width / 2, width / 2, -width / 2, width / 2))
#                             vmin=vmin, vmax=vmax)#, norm = norm)
    # filename="{}/{}_{:03d}".format(foldername, SIMNUMBER, i));
    ax.set_title(gas_img.title, fontsize=12)

In [30]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(i, gas_images):
    plot_gas(gas_images[i])

In [31]:
play = widgets.Play(
    interval=1000,
    value=0,
    min=0,
    max=len(imgs),
    step=1,
    description="Press play",
    disabled=False
)

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(imgs)-1,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
widgets.jslink((play, 'value'), (slider, 'value'))
h = widgets.HBox([play, slider])

In [32]:
w = interactive(f, i=slider, gas_images=fixed(imgs))
hb = widgets.HBox([play, slider])
widgets.VBox([hb, w])

A Jupyter Widget

In [25]:
# foldername = "pngs_{}".format(SIMNUMBER) 
# os.makedirs(foldername, exist_ok=True)
# for i, sim in enumerate(load_moria(SIMNUMBER)):
#     sim.properties.pop('boxsize')
#     sim.g['smooth'] /= 2
#     pynbody.analysis.halo.center(sim, shrink_factor=0.9)
#     fig, ax = plt.subplots(figsize=(10,10))
#     sph.image(sim.g, qty="rho", resolution=400, units="g cm^-2",
#               width=10,# vmin=2e-1, vmax=5e-4,
#               filename="{}/{}_{:03d}".format(foldername, SIMNUMBER, i));
#     time = '$t={%5.2f}$' % sim.properties['time'].in_units("Gyr")
#     ax = plt.gca()
#     ax.set_title(time, fontsize=12)
#     print(i)
#     plt.close(fig)